In [1]:
# Utils
import os, sys, re, time, gc, types, string, unicodedata, unidecode, string, warnings, inspect

import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt

import numpy as np
import time
import pickle

from sklearn.metrics import accuracy_score
from IPython.display import display, Markdown, Latex

import shutil

from tensorflow.contrib.learn import DNNClassifier
import scikitplot as skplt

import scipy
#plt.style.use('fivethirtyeight')

tf.set_random_seed(0)


/root/anaconda3/envs/tensorflow/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


# Parameters

In [2]:
root_path = "../"
path_model = root_path + 'models/testV2'
#features_path = root_path + 'data/features/data_tfid_hash28_n1000_SVD2.p'
#features_path = root_path + 'data/features/data_tfid_hash28_n10000_bigram_svd1000.p'
#features_path = root_path + 'data/features/data_tfid_hash28_n1000.p'

#features_path = root_path + 'data/features/dataV2_tfid_hash28_n10000_svd1000_sin_stemmig.p'
features_path = root_path + 'data/features/dataV2_tfid_hash28_n10000_svd1000_bigram_sin_stemmig.p'
features_path = root_path + 'data/features/dataV2_tfid_hash28_n10000_svd1000_bigram_con_stemmig.p'





# Load Data

### Output

In [3]:
X ,y1 ,y2 , y3 = pickle.load( open( features_path, "rb" ) )


In [4]:

# fraction = 0.1
# np.random.seed(42)
# indices = np.random.choice(X.shape[0], round(fraction*X.shape[0]), replace=False)

# X = X[indices]
# y1 = y1[indices]

# X.shape[0]

In [5]:
train_fraction = 0.70

y1 = y1.values    

np.random.seed(42)
train_indices = np.random.choice(X.shape[0], round(train_fraction*X.shape[0]), replace=False)
test_indices = np.array(list(set(range(X.shape[0])) - set(train_indices)))

X_train = X[train_indices]
y_train = y1[train_indices]
X_test = X[test_indices]
y_test = y1[test_indices]


In [6]:
labels = list(set(y_train))
labels.sort()
nClasses = len(labels)

def calcWeights(y):
    sizes = {label: y_train[y_train == label].shape[0] for label in labels}
    weights = np.asarray([float(len(y_train))/(sizes[label]*nClasses) for label in y]) #n_samples / (n_classes * np.bincount(y))
    weights = np.power(weights,1) # 1.4
    return weights 

def calcWeightsL(y):
    #lerko
    scale_factor = 10e3
    sizes = {label: y_train[y_train == label].shape[0] for label in labels}
    weights = np.asarray([scale_factor/sizes[label] for label in y])
    return weights 

weights_train = calcWeights(y_train)
weights_test = calcWeights(y_test)

# Model

### DNN graph generation

In [7]:
epochs = 20000

if isinstance(X_train, scipy.sparse.csr.csr_matrix):
    X_train = X_train.todense()
    X_test = X_test.todense()

# Define the test inputs
def get_train_inputs():    
    dataset = tf.estimator.inputs.numpy_input_fn({'x': X_train,'class_weights': weights_train},
                                                  y_train[:,np.newaxis],
                                                  shuffle=True,
                                                  batch_size=128, #min 64 , 1024
                                                  num_epochs=epochs)
    return dataset

def get_test_inputs():
    dataset = tf.estimator.inputs.numpy_input_fn({'x': X_test,'class_weights': weights_test},
                                                  y_test[:,np.newaxis],
                                                  shuffle=False)
    return dataset

### functions graph tensorflow

In [8]:



feature_columns = [tf.contrib.layers.real_valued_column('x', dimension=X_train.shape[1])]

optimizer = tf.train.ProximalAdagradOptimizer(
                              learning_rate=10.0,
                              l1_regularization_strength= 0.0#0.0001
                              )

classifier = DNNClassifier(                                
                           n_classes=len(labels), label_keys=labels, feature_columns=feature_columns,
                           hidden_units=[3000,2000], #2000, 1000, 100
                           dropout=0.5,
                           #optimizer = optimizer,
                           weight_column_name='class_weights',
                           model_dir = path_model,
                           config = tf.contrib.learn.RunConfig(save_checkpoints_steps = 4096,
                           save_checkpoints_secs = None)                           
                          )

Instructions for updating:
When switching to tf.estimator.Estimator, use tf.estimator.RunConfig instead.
Instructions for updating:
Please switch to tf.contrib.estimator.*_head.
Instructions for updating:
Please replace uses of any Estimator from tf.contrib.learn with an Estimator from tf.estimator.*
INFO:tensorflow:Using config: {'_device_fn': None, '_environment': 'local', '_task_id': 0, '_log_step_count_steps': 100, '_session_config': None, '_num_worker_replicas': 0, '_evaluation_master': '', '_tf_random_seed': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fe454079780>, '_master': '', '_num_ps_replicas': 0, '_save_checkpoints_steps': 4096, '_keep_checkpoint_every_n_hours': 10000, '_save_summary_steps': 100, '_keep_checkpoint_max': 5, '_task_type': None, '_save_checkpoints_secs': None, '_is_chief': True, '_train_distribute': None, '_model_dir': '../models/testV2'}


# Training



In [9]:
tf.set_random_seed(0)

delete_old_model = True
if delete_old_model:
    try:
        os.system("rm -rf "+path_model)
        os.system("mkdir "+path_model)
    except:
        print("error")
    
validation_monitor = tf.contrib.learn.monitors.ValidationMonitor(
    #input_fn= get_train_inputs(),
    input_fn= get_test_inputs(),
    
    every_n_steps=500,
    #early_stopping_metric="accuracy",   #accurray mucho mas rapido,7s vs 40s, pero 2% peor
    early_stopping_metric="loss",   
    early_stopping_metric_minimize=True,
    early_stopping_rounds=2500)

start = time.time()
print("start")

#classifier.fit(input_fn=get_train_inputs(), monitors=[validation_monitor], steps=epochs, max_steps=None)
classifier.fit(input_fn=get_train_inputs(), monitors=[validation_monitor], steps=epochs, max_steps=None)

end = time.time()
print("Training time :" + str(end - start) )


Instructions for updating:
Monitors are deprecated. Please use tf.train.SessionRunHook.
start
Instructions for updating:
When switching to tf.estimator.Estimator, use tf.estimator.EstimatorSpec. You can use the `estimator_spec` method to create an equivalent one.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into ../models/testV2/model.ckpt.
INFO:tensorflow:loss = 2.4736545, step = 0
INFO:tensorflow:global_step/sec: 190.211
INFO:tensorflow:loss = 2.223821, step = 100 (0.527 sec)
INFO:tensorflow:global_step/sec: 233.919
INFO:tensorflow:loss = 1.6530454, step = 200 (0.427 sec)
INFO:tensorflow:global_step/sec: 235.658
INFO:tensorflow:loss = 1.9713886, step = 300 (0.424 sec)
INFO:tensorflow:global_step/sec: 232.423
INFO:tensorflow:loss = 1.7306763, step = 400 (0.430 sec)
INFO:tensorflow:global_step/sec: 234.24
INFO:tensorflow:Startin

INFO:tensorflow:global_step/sec: 233.942
INFO:tensorflow:loss = 0.46968395, step = 5900 (0.427 sec)
INFO:tensorflow:global_step/sec: 237.901
INFO:tensorflow:loss = 0.29080373, step = 6000 (0.420 sec)
INFO:tensorflow:global_step/sec: 234.714
INFO:tensorflow:loss = 0.37454185, step = 6100 (0.426 sec)
INFO:tensorflow:global_step/sec: 235.387
INFO:tensorflow:loss = 0.6898747, step = 6200 (0.425 sec)
INFO:tensorflow:global_step/sec: 234.507
INFO:tensorflow:loss = 0.34732816, step = 6300 (0.426 sec)
INFO:tensorflow:global_step/sec: 232.947
INFO:tensorflow:loss = 0.2736777, step = 6400 (0.429 sec)
INFO:tensorflow:global_step/sec: 233.736
INFO:tensorflow:loss = 0.22655912, step = 6500 (0.428 sec)
INFO:tensorflow:global_step/sec: 235.13
INFO:tensorflow:loss = 0.27735323, step = 6600 (0.425 sec)
INFO:tensorflow:global_step/sec: 231.333
INFO:tensorflow:loss = 0.38442695, step = 6700 (0.432 sec)
INFO:tensorflow:global_step/sec: 233.167
INFO:tensorflow:loss = 0.43579835, step = 6800 (0.429 sec)
INF

INFO:tensorflow:loss = 0.31247774, step = 12700 (0.425 sec)
INFO:tensorflow:global_step/sec: 234.291
INFO:tensorflow:loss = 0.21606109, step = 12800 (0.427 sec)
INFO:tensorflow:global_step/sec: 232.753
INFO:tensorflow:loss = 0.28526857, step = 12900 (0.430 sec)
INFO:tensorflow:global_step/sec: 234.767
INFO:tensorflow:loss = 0.43727154, step = 13000 (0.426 sec)
INFO:tensorflow:global_step/sec: 234.502
INFO:tensorflow:loss = 0.3772918, step = 13100 (0.426 sec)
INFO:tensorflow:global_step/sec: 230.262
INFO:tensorflow:loss = 0.5009268, step = 13200 (0.434 sec)
INFO:tensorflow:global_step/sec: 235.203
INFO:tensorflow:loss = 0.20346725, step = 13300 (0.425 sec)
INFO:tensorflow:global_step/sec: 233.335
INFO:tensorflow:loss = 0.5050606, step = 13400 (0.429 sec)
INFO:tensorflow:global_step/sec: 234.46
INFO:tensorflow:loss = 0.14887848, step = 13500 (0.428 sec)
INFO:tensorflow:global_step/sec: 233.96
INFO:tensorflow:loss = 0.22293529, step = 13600 (0.426 sec)
INFO:tensorflow:global_step/sec: 235

# Evaluation performance


In [16]:
import tensorflow.contrib.slim as slim


def model_summary():
    model_vars = tf.trainable_variables()
    slim.model_analyzer.analyze_vars(model_vars, print_info=True)

model_summary()

---------
Variables: name (type shape) [size]
---------
Total size of variables: 0
Total bytes of variables: 0


In [10]:
#X_test = X_test.todense()

def input_fn_evaluate():
    dataset = {'x': tf.constant(X_test )}    
    return dataset

pred_test = classifier.predict_classes(input_fn=input_fn_evaluate)
y_test_hat = np.asarray([x.decode('UTF-8') for x in list(pred_test)])
y_test_hat = y_test_hat.astype(str)


acc = accuracy_score(y_true=y_test, y_pred=y_test_hat)
cfm = skplt.metrics.confusion_matrix(y_test, y_test_hat)
cfm = cfm/cfm.astype(np.float).sum(axis=1)
mdp = np.mean(cfm.diagonal())

skplt.metrics.plot_confusion_matrix(y_test, y_test_hat,normalize='True')
plt.xticks(rotation=45)
plt.show()

#print('Accuracy in test: {}'.format(acc))
display(Markdown('## Mean diag. principal confusion Matrix : {} '.format(mdp*100)))

display(Markdown('## Accuracy in test: {} '.format(acc*100)))

INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ../models/testV2/model.ckpt-20000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


KeyboardInterrupt: 

In [ ]:
#X_test = X_test.todense()

def input_fn_evaluate():
    dataset = {'x': tf.constant(X_test )}    
    return dataset

pred_test = classifier.predict_classes(input_fn=input_fn_evaluate)
y_test_hat = np.asarray([x.decode('UTF-8') for x in list(pred_test)])
y_test_hat = y_test_hat.astype(str)


acc = accuracy_score(y_true=y_test, y_pred=y_test_hat)
cfm = skplt.metrics.confusion_matrix(y_test, y_test_hat)
cfm = cfm/cfm.astype(np.float).sum(axis=1)
mdp = np.mean(cfm.diagonal())

skplt.metrics.plot_confusion_matrix(y_test, y_test_hat,normalize='True')
plt.xticks(rotation=45)
plt.show()

#print('Accuracy in test: {}'.format(acc))
display(Markdown('## Mean diag. principal confusion Matrix : {} '.format(mdp*100)))

display(Markdown('## Accuracy in test: {} '.format(acc*100)))

In [ ]:
#X_test = X_test.todense()

def input_fn_evaluate():
    dataset = {'x': tf.constant(X_test )}    
    return dataset

pred_test = classifier.predict_classes(input_fn=input_fn_evaluate)
y_test_hat = np.asarray([x.decode('UTF-8') for x in list(pred_test)])
y_test_hat = y_test_hat.astype(str)


acc = accuracy_score(y_true=y_test, y_pred=y_test_hat)
cfm = skplt.metrics.confusion_matrix(y_test, y_test_hat)
cfm = cfm/cfm.astype(np.float).sum(axis=1)
mdp = np.mean(cfm.diagonal())

skplt.metrics.plot_confusion_matrix(y_test, y_test_hat,normalize='True')
plt.xticks(rotation=45)
plt.show()

#print('Accuracy in test: {}'.format(acc))
display(Markdown('## Mean diag. principal confusion Matrix : {} '.format(mdp*100)))

display(Markdown('## Accuracy in test: {} '.format(acc*100)))


### save models

In [ ]:

pickle.dump( (path_model,labels), open( path_model + "/info_model.p", "wb" ) )

In [ ]:
# def input_fn_evaluate1():
#     dataset = {'x': tf.constant(X_test[0:25000,:].todense() )}    
#     return dataset

# pred_test1 = classifier.predict_classes(input_fn=input_fn_evaluate1)

# def input_fn_evaluate2():
#     dataset = {'x': tf.constant(X_test[25000:,:].todense() )}    
#     return dataset
# pred_test2 = classifier.predict_classes(input_fn=input_fn_evaluate2)50


In [ ]:
# y_test_hat1 = np.asarray([x.decode('UTF-8') for x in list(pred_test1)])
# y_test_hat2 = np.asarray([x.decode('UTF-8') for x in list(pred_test2)])
# y_test_hat = np.concatenate((y_test_hat1, y_test_hat2), axis=0)

In [ ]:

# y_test_hat = y_test_hat.astype(str)


# acc = accuracy_score(y_true=y_test, y_pred=y_test_hat)

# skplt.metrics.plot_confusion_matrix(y_test, y_test_hat,normalize='True')
# plt.xticks(rotation=45)
# plt.show()

# #print('Accuracy in test: {}'.format(acc))
# display(Markdown('## Accuracy in test: {} '.format(acc*100)))